In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [18]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2014-01-01'

btc_df = yf.download('BTC-USD',start_date, today)

btc_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,$465.86,$468.17,$452.42,$457.33,$457.33,21056800
2014-09-18,$456.86,$456.86,$413.10,$424.44,$424.44,34483200
2014-09-19,$424.10,$427.83,$384.53,$394.80,$394.80,37919700
2014-09-20,$394.67,$423.30,$389.88,$408.90,$408.90,36863600
2014-09-21,$408.08,$412.43,$393.18,$398.82,$398.82,26580100


In [19]:
btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2441 entries, 2014-09-17 to 2021-05-27
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2441 non-null   float64
 1   High       2441 non-null   float64
 2   Low        2441 non-null   float64
 3   Close      2441 non-null   float64
 4   Adj Close  2441 non-null   float64
 5   Volume     2441 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 133.5 KB


In [20]:
btc_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [21]:
btc_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [22]:
btc_df.reset_index(inplace=True)
btc_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [23]:
df = btc_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [24]:
df.tail()

,ds,y
2436,2021-05-23,"$37,531.45"
2437,2021-05-24,"$34,700.36"
2438,2021-05-25,"$38,795.78"
2439,2021-05-26,"$38,392.62"
2440,2021-05-27,"$39,197.78"


In [25]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of BTC Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [26]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [27]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2801,2022-05-23
2802,2022-05-24
2803,2022-05-25
2804,2022-05-26
2805,2022-05-27


In [28]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2801,2022-05-23,"$67,385.52","$44,214.60","$92,502.84"
2802,2022-05-24,"$66,127.02","$44,615.61","$90,577.25"
2803,2022-05-25,"$64,520.87","$42,756.75","$88,573.85"
2804,2022-05-26,"$62,889.99","$41,209.34","$86,233.06"
2805,2022-05-27,"$61,451.31","$39,881.76","$84,283.91"


In [29]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

35019.833240257496

In [30]:
#check here for real and predicted
plot_plotly(m, forecast)

In [31]:
plot_components_plotly(m, forecast)